In [1]:
import pandas as pd
import numpy as np

collection = pd.read_csv("tate/collection.csv", index_col=0)
details = pd.read_csv("tate/details.csv", index_col=0)

collection = collection.join(details).drop(['highres url', 'thumb_url'], axis=1)

collection

,artist,title,year,url,thumb,artist years,medium,dimensions
N01506,"Sir John Everett Millais, Bt",Ophelia,1851-2,millais-ophelia-n01506,N01506_7.jpg,1829-1896,Oil paint on canvas,Support: 762 x 1118 mmframe: 1105 x 1458 x 145 mm
T07502,Chris Ofili,"No Woman, No Cry",1998,ofili-no-woman-no-cry-t07502,T07502_7.jpg,1968,"Oil paint, acrylic paint, graphite, polyester ...",Support: 2438 x 1828 x 51 mm
T03254,David Hockney,A Bigger Splash,1967,hockney-a-bigger-splash-t03254,T03254_7.jpg,1937,Acrylic paint on canvas,Support: 2425 x 2439 x 30 mm
T13896,John Constable,Salisbury Cathedral from the Meadows,exhibited 1831,constable-salisbury-cathedral-from-the-meadows...,T13896_7.jpg,1776-1837,Oil paint on canvas,Support: 1537 x 1920 mmframe: 1800 x 2187 x 10...
T07573,Marcel Duchamp,Fountain,"1917, replica 1964",duchamp-fountain-t07573,T07573_7.jpg,1887-1968,Porcelain,Unconfirmed: 360 x 480 x 610 mm
...,...,...,...,...,...,...,...,...
T15193,Gulsun Karamustafa,Prison Paintings 14,1972,karamustafa-prison-paintings-14-t15193,T15193_7.jpg,1946,"Acrylic paint, graphite, crayon, and ink on paper",Support: 425 x 260 mm
T15191,Gulsun Karamustafa,Prison Paintings 11,1972,karamustafa-prison-paintings-11-t15191,T15191_7.jpg,1946,"Acrylic paint, graphite, crayon, and ink on paper",Support: 294 x 435 mm
T15190,Gulsun Karamustafa,Prison Paintings 10,1972,karamustafa-prison-paintings-10-t15190,T15190_7.jpg,1946,"Acrylic paint, graphite, crayon, and ink on paper",Support: 360 x 447 mm
T15143,Claudette Johnson,Standing Figure with African Masks,2018,johnson-standing-figure-with-african-masks-t15143,T15143_7.jpg,1959,Pastel and gouache on paper,Support: 1515 x 1220 mm


In [2]:
collection.loc['N03136']

artist                                      Richard Wilson
title                       The Cock Tavern, Cheam, Surrey
year                                                c.1745
url             wilson-the-cock-tavern-cheam-surrey-n03136
thumb                                         N03136_7.jpg
artist years                                     1713-1782
medium                                 Oil paint on canvas
dimensions                           Support: 730 x 436 mm
Name: N03136, dtype: object

In [3]:
def parse_year(s):
    s = s.lower()
    if s[:2] == 'c.':
        s = s[2:]
    if s[:3] == 'ci.':
        s = s[3:]
    if s[0] == '?':
        s = s[1:]

    toks = s.split()
    while toks[0] in ['c.', 'ci.', 'exhibited', '?exhibited', 'after', 'engraved', 'late', 'early', 
                      'published', 'first', 'printed', 'by', 'before']:
        toks = toks[1:]
    
    s = ' '.join(toks)
    
    if s[:2] == 'c.':
        s = s[2:]
    if s[0] == '?':
        s = s[1:]    
    
    if '-' in s:
        s = s.split('-')[0]
    
    try:
        return int(s)
    except:
        pass
    
    try:
        return int(s[:4])
    except:
        return np.nan
    
    
def parse_dimensions(s):    
    raw = str(s)
    s = s.lower().replace('mm', '')
    try:
        if s.startswith('support'):
            toks = s.split(':')
#             s = toks[-1].strip()
            if len(toks) >= 2:
                s = toks[1].strip()
            else:
                s = toks[0].strip()
        else:
            s = s.split(':')[1].strip()
    except:
#         print("1>>", s)
        return None, None # There's a handful of bad strings here
        
    s = s.replace(':', '')
    for w in ['unconfirmed', 'displayed', 'framed', 'frame', 'support', 'object']:
        s = s.replace(w, '').strip()
    s = s.replace('×', 'x') # Argh

    try:
        w,h,*_ = s.split(' x ')
        return int(float(w)), int(float(h))
    except: 
#         print("2>>", raw)
#         print("   ", s, s.split(' x '))
        return None, None # There's a handful of bad strings here
    
def parse_width(s):
    return parse_dimensions(s)[0]

def parse_height(s):
    return parse_dimensions(s)[1]    
    
collection['parsed'] = collection['year'].apply(parse_year)
collection['result'] = collection['parsed'].apply(lambda s: 'NaN' if np.isnan(s) else str(int(s)))

collection

,artist,title,year,url,thumb,artist years,medium,dimensions,parsed,result
N01506,"Sir John Everett Millais, Bt",Ophelia,1851-2,millais-ophelia-n01506,N01506_7.jpg,1829-1896,Oil paint on canvas,Support: 762 x 1118 mmframe: 1105 x 1458 x 145 mm,1851.0,1851
T07502,Chris Ofili,"No Woman, No Cry",1998,ofili-no-woman-no-cry-t07502,T07502_7.jpg,1968,"Oil paint, acrylic paint, graphite, polyester ...",Support: 2438 x 1828 x 51 mm,1998.0,1998
T03254,David Hockney,A Bigger Splash,1967,hockney-a-bigger-splash-t03254,T03254_7.jpg,1937,Acrylic paint on canvas,Support: 2425 x 2439 x 30 mm,1967.0,1967
T13896,John Constable,Salisbury Cathedral from the Meadows,exhibited 1831,constable-salisbury-cathedral-from-the-meadows...,T13896_7.jpg,1776-1837,Oil paint on canvas,Support: 1537 x 1920 mmframe: 1800 x 2187 x 10...,1831.0,1831
T07573,Marcel Duchamp,Fountain,"1917, replica 1964",duchamp-fountain-t07573,T07573_7.jpg,1887-1968,Porcelain,Unconfirmed: 360 x 480 x 610 mm,1917.0,1917
...,...,...,...,...,...,...,...,...,...,...
T15193,Gulsun Karamustafa,Prison Paintings 14,1972,karamustafa-prison-paintings-14-t15193,T15193_7.jpg,1946,"Acrylic paint, graphite, crayon, and ink on paper",Support: 425 x 260 mm,1972.0,1972
T15191,Gulsun Karamustafa,Prison Paintings 11,1972,karamustafa-prison-paintings-11-t15191,T15191_7.jpg,1946,"Acrylic paint, graphite, crayon, and ink on paper",Support: 294 x 435 mm,1972.0,1972
T15190,Gulsun Karamustafa,Prison Paintings 10,1972,karamustafa-prison-paintings-10-t15190,T15190_7.jpg,1946,"Acrylic paint, graphite, crayon, and ink on paper",Support: 360 x 447 mm,1972.0,1972
T15143,Claudette Johnson,Standing Figure with African Masks,2018,johnson-standing-figure-with-african-masks-t15143,T15143_7.jpg,1959,Pastel and gouache on paper,Support: 1515 x 1220 mm,2018.0,2018


In [4]:
idx = (collection['year'] != collection['result']) \
                & (collection['result'] == "NaN")  \
                & (~collection['year'].isin(["date not known", "n.d", "no date"]))
collection.loc[idx,['year', 'result']]

,year,result
T12751,exhibition copy 2007 (original 1993),NaN
AR01214,"mid-1970s, printed 2013",NaN
N02985,late 18th C,NaN
T14452,mid-1980s,NaN
N02716,late 18th C,NaN
N02981,late 18th C,NaN
T07526,?mid 1830s or later,NaN
T07525,?mid 1830s or later,NaN


In [5]:
idx = (collection['year'] != collection['result']) & (collection['result'] != "NaN")
collection.loc[idx,['year', 'result']]

,year,result
N01506,1851-2,1851
T13896,exhibited 1831,1831
T07573,"1917, replica 1964",1917
N01615,1885-6,1885
N00530,exhibited 1842,1842
...,...,...
P14819,"1929, later print",1929
D22162,?1831,1831
D24854,c.1830,1830
P81976,1980-9,1980


In [6]:
# Cleanup: remove parse, result and write into years.
# then drop rows without years.
collection['year'] = collection['parsed']

collection = collection.drop(['parsed', 'result'], axis=1)

collection = collection[~collection['year'].isna()] # Remove rows w/o parseable years
collection['year'] = collection['year'].astype(int)

In [7]:
# Restrict to paintings, drop url and thumb columns.
collection = collection[collection['medium'].str.contains('on canvas', na=False)]
collection = collection.drop(['url', 'thumb', 'medium'], axis=1)
collection

,artist,title,year,artist years,dimensions
N01506,"Sir John Everett Millais, Bt",Ophelia,1851,1829-1896,Support: 762 x 1118 mmframe: 1105 x 1458 x 145 mm
T07502,Chris Ofili,"No Woman, No Cry",1998,1968,Support: 2438 x 1828 x 51 mm
T03254,David Hockney,A Bigger Splash,1967,1937,Support: 2425 x 2439 x 30 mm
T13896,John Constable,Salisbury Cathedral from the Meadows,1831,1776-1837,Support: 1537 x 1920 mmframe: 1800 x 2187 x 10...
T00897,Roy Lichtenstein,Whaam!,1963,1923-1997,Support: 1727 x 4064 mmframe: 1747 x 4084 x 60 mm
...,...,...,...,...,...
T14823,Unknown artist,Leon Trotsky,1980,artist,Support: 510 x 480 mm
P02269,Peter Phillips,Untitled Composition II,1974,1939,Unconfirmed: 698 x 999 mm
AL00397,Louise Bourgeois,Untitled,1946,1911-2010,Support: 660 x 1116 x 20 mmframe: 770 x 1224 x...
T14824,Unknown artist,Leon Trotsky,1980,artist,Support: 638 x 511 mm


In [8]:
# Parse ground truth from labels.txt
from pathlib import Path
import os

data = []
index = []
with open('tate/labels.txt', 'r') as f:
    for l in f:
        path, label = l.rstrip().split()
        _, name = os.path.split(path)
        name, _ = os.path.splitext(name)
        data.append(label)
        index.append(name)
        
ser = pd.Series(data=data, index=index)

collection['groundtruth'] = ser
collection = collection.dropna()
collection

,artist,title,year,artist years,dimensions,groundtruth
N01506,"Sir John Everett Millais, Bt",Ophelia,1851,1829-1896,Support: 762 x 1118 mmframe: 1105 x 1458 x 145 mm,O
T13896,John Constable,Salisbury Cathedral from the Meadows,1831,1776-1837,Support: 1537 x 1920 mmframe: 1800 x 2187 x 10...,L
T05010,Pablo Picasso,Weeping Woman,1937,1881-1973,Support: 608 x 500 mmframe: 847 x 739 x 86 mm,O
N01615,John Singer Sargent,"Carnation, Lily, Lily, Rose",1885,1856-1925,Support: 1740 x 1537 mmframe: 2185 x 1970 x 13...,O
T02343,Salvador Dalí,Metamorphosis of Narcissus,1937,1904-1989,Support: 511 x 781 mmframe: 820 x 1092 x 85 mm,O
...,...,...,...,...,...,...
N05609,Maurice Sterne,Mexican Church Interior,1934,1878-1957,Support: 1283 x 1022 mm,O
T14823,Unknown artist,Leon Trotsky,1980,artist,Support: 510 x 480 mm,P
AL00397,Louise Bourgeois,Untitled,1946,1911-2010,Support: 660 x 1116 x 20 mmframe: 770 x 1224 x...,O
T14824,Unknown artist,Leon Trotsky,1980,artist,Support: 638 x 511 mm,P


In [9]:
# Subsample 'O' class to make our examples work nicely
ix = collection['groundtruth'] == 'O'

np.random.seed(42)
jx = np.multiply(ix, np.random.rand(*ix.shape))
jx[~ix] = 1
jx = jx > 0.604

display(collection[jx]['groundtruth'].value_counts())

collection = collection[jx]

O    775
P    723
L    670
Name: groundtruth, dtype: int64

In [10]:

dims = collection['dimensions'].apply(parse_dimensions).apply(pd.Series)
dims.columns = ['height', 'width']

collection = collection.join(dims)

collection = collection[~collection['width'].isna()]
collection['width'] = collection['width'].astype(int)
collection['height'] = collection['height'].astype(int)
collection = collection.drop(['dimensions'], axis=1)

In [11]:
collection = collection.drop(['artist years'], axis=1)

In [12]:
display(collection.drop(['groundtruth'], axis=1))

,artist,title,year,height,width
T13896,John Constable,Salisbury Cathedral from the Meadows,1831,1537,1920
T05010,Pablo Picasso,Weeping Woman,1937,608,500
N05915,Pablo Picasso,Bust of a Woman,1909,727,600
N00530,Joseph Mallord William Turner,Snow Storm - Steam-Boat off a Harbour’s Mouth,1842,914,1219
T00598,Richard Dadd,The Fairy Feller’s Master-Stroke,1855,540,394
...,...,...,...,...,...
N05609,Maurice Sterne,Mexican Church Interior,1934,1283,1022
T14823,Unknown artist,Leon Trotsky,1980,510,480
AL00397,Louise Bourgeois,Untitled,1946,660,1116
T14824,Unknown artist,Leon Trotsky,1980,638,511


In [13]:
collection.loc['N03136']

artist                         Richard Wilson
title          The Cock Tavern, Cheam, Surrey
year                                     1745
groundtruth                                 L
height                                    730
width                                     436
Name: N03136, dtype: object

In [14]:
from PIL import Image
import os


for i, row in collection.iterrows():
    file = '/home/felix/Development/Tate/paintings/{}.jpg'.format(i)
    if not os.path.exists(file):
        continue
        
   
    im = Image.open(file)
    width, height = im.size
    rimg = width/height
    rdata = row['width']/row['height']
    
    if abs(rimg - rdata) > abs(1.0/rimg - rdata):
        print("Swapping dimensions of {}".format(i))
        width, height = row['width'], row['height']
        collection.loc[i,'width'] = height
        collection.loc[i,'height'] = width


Swapping dimensions of T01972
Swapping dimensions of N00529
Swapping dimensions of T03374
Swapping dimensions of N01511
Swapping dimensions of T07403
Swapping dimensions of AR00344
Swapping dimensions of T01772
Swapping dimensions of T00096
Swapping dimensions of T00330
Swapping dimensions of T12309
Swapping dimensions of T11915
Swapping dimensions of T13803
Swapping dimensions of T14504
Swapping dimensions of T00605
Swapping dimensions of T07980
Swapping dimensions of T03191
Swapping dimensions of T06666
Swapping dimensions of T00523
Swapping dimensions of AR00031
Swapping dimensions of T00720
Swapping dimensions of T07289
Swapping dimensions of T14996
Swapping dimensions of T01069
Swapping dimensions of N04565
Swapping dimensions of T00802
Swapping dimensions of T13272
Swapping dimensions of T03350
Swapping dimensions of N05668
Swapping dimensions of T15024
Swapping dimensions of N05945
Swapping dimensions of N02974
Swapping dimensions of T00907
Swapping dimensions of T12973
Swapping

In [15]:
collection.to_csv('tate/paintings.csv')